In [ ]:
import data_modules
import torch
from data_modules.baseline_finetune_dm import BaselineFinetuneDM
from data_modules.generic_finetune_dm import GenericFinetuneDM

dm = GenericFinetuneDM(train_dir="data/topex-printer/train/", test_dir="data/topex-printer/test/")
# gftdm = GenericFinetuneDM()
dm.prepare_data()
dm.setup()
train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

inputs, labels = next(iter(train_dataloader))
print(dm.num_classes)
print(len(dm.test.classes))
print(dm.test.classes == dm.num_classes)

In [ ]:
import os
import shutil

import pandas as pd

data = "/home/dennis/projects/evaluation-pipeline/data/datasets/visda2017_meshes/train"
out = "/home/dennis/projects/evaluation-pipeline/data/datasets/visda2017_meshgrid"

d = {}
for path, dns, fns in os.walk(data):
    for fn in fns:
        fnsplit = fn.split("__")
        mesh = fnsplit[0]
        angles = fnsplit[1].split("_")
        light_angle = angles[1]
        if f"{mesh}_{light_angle}" in d.keys():
            d[f"{mesh}_{light_angle}"].append(fn)
        else:
            d[f"{mesh}_{light_angle}"] = [fn]

In [ ]:
import os
import random

import torchvision
from PIL import Image
from torchvision import transforms as transforms
from torchvision.io import read_image
from torchvision.utils import make_grid


def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result


img_dir = "../data/modelnet10_toy/train/chair/chair_0001"

grid_size = (4, 4)
padding = 0
margin = 16
n_img = grid_size[0] * grid_size[1]
renders = os.listdir(img_dir)
# rnd_model = random.sample(renders, k=1)[0]
# rnd_model = rnd_model.split("_")[3]
# renders = [render for render in renders if render.split("_")[3] == rnd_model]
rnd_imgs = random.sample(renders, k=n_img)
imgs = [read_image(f"{img_dir}/{img_fn}") for img_fn in rnd_imgs]
grid = make_grid(imgs, nrow=grid_size[1], padding=padding)
img_grid = torchvision.transforms.ToPILImage()(grid)
img_grid = img_grid.resize((224, 224))
img_grid = add_margin(img_grid, margin, margin, margin, margin, (0, 0, 0))
img_grid

In [ ]:
import os

import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

# Define the mean and standard deviation values from the ImageNet dataset
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

# Define a transform pipeline that applies any necessary image transformations and normalization
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

# Load an image from file (in this case, a JPEG file)
dir = "../data/visda2017/train"
for path, dir, fn in os.walk(dir):
    for f in fn:
        image = Image.open(f"{path}/{f}")
        print(f"minmax original: {np.array(image).min()}, {np.array(image).max()}")
        # Apply the transform pipeline to the image
        transformed_image = transform(image)
        # Print the minimum and maximum pixel values in the transformed image tensor
        print(f"minmax transformed: {transformed_image.min()}, {transformed_image.max()}")

### DataModule testing

In [ ]:
import torch
from torchvision import transforms

from src.datamodules.generic_finetune_dm import GenericFinetuneDM

dm = GenericFinetuneDM(
    train_dir="data/visda2017/train/",
    test_dir="data/visda2017/test/",
    augmix=True,
    random_color_jitter=False,
    toy=False,
)
# gftdm = GenericFinetuneDM()
dm.prepare_data()
dm.setup()
train_dataloader = dm.train_dataloader()

# print(train_dataloader.dataset.transform)

inputs, labels = next(iter(train_dataloader))
topil = transforms.ToPILImage()
imgs = [topil(img) for img in inputs]
imgs[1]

### generic_adaptation_dm Datamodule


In [ ]:
import torch
from IPython.display import display
from torchvision import transforms

from src.datamodules.generic_adaptation_dm import GenericAdaptationDM

dm = GenericAdaptationDM(
    train_src_dirs=["data/visda2017/train/"],
    train_target_dirs=["data/visda2017/test/"],
    val_dirs=["data/visda2017/val/"],
    test_dirs=["data/visda2017/test/"],
    augmix=True,
    random_color_jitter=False,
    toy=True,
    num_workers=1,
)
dm.prepare_data()
dm.setup()
train_dataloader = dm.train_dataloader()

src, target = next(iter(train_dataloader))
src_img, src_label = src
target_img, target_label = target
src_img = [transforms.ToPILImage()(img) for img in src_img]
target_img = [transforms.ToPILImage()(img) for img in target_img]
display(src_img[0])
print("train_src:", dm.idx2label[src_label[0].item()])
display(target_img[0])
print("train_target:", dm.idx2label[target_label[0].item()])

val_dataloader = dm.val_dataloader()
val_img, val_label = next(iter(val_dataloader))
val_img = [transforms.ToPILImage()(img) for img in val_img]
display(val_img[0])
print("val:", dm.idx2label[val_label[0].item()])

test_dataloader = dm.test_dataloader()
test_img, test_label = next(iter(test_dataloader))
test_img = [transforms.ToPILImage()(img) for img in test_img]
display(test_img[0])
print("test:", dm.idx2label[test_label[0].item()])